In [1]:
import scipy.io
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D, MaxPool2D, Flatten
from keras import optimizers
import keras.utils as np_utils
from keras import backend as K
import matplotlib.pyplot as plt
import sys
import urllib
import os

Using TensorFlow backend.


In [2]:
####### download mnist file from website 
if not os.path.isfile("train-images-idx3-ubyte.gz"):
    urllib.request.urlretrieve("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz","train-images-idx3-ubyte.gz")
if not os.path.isfile("train-labels-idx1-ubyte.gz"):
    urllib.request.urlretrieve("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz","train-labels-idx1-ubyte.gz")

######## reading from the .gz files
import gzip
f=gzip.open("train-images-idx3-ubyte.gz",'rb')
buf=f.read(4)
magic_num=np.frombuffer(buf,dtype=np.dtype(np.int32).newbyteorder('>'))
if magic_num != 2051:
    print("invalid file ... exiting...")
    exit()
buf=f.read(4)
N=np.frombuffer(buf,dtype=np.dtype(np.int32).newbyteorder('>'))[0]
buf=f.read(4)
H=np.frombuffer(buf,dtype=np.dtype(np.int32).newbyteorder('>'))[0]
buf=f.read(4)
W=np.frombuffer(buf,dtype=np.dtype(np.int32).newbyteorder('>'))[0]
train_img=np.frombuffer(f.read(H*W*N),dtype=np.uint8)
train_img=train_img.reshape(N,H,W,1)
f.close()

f=gzip.open("train-labels-idx1-ubyte.gz",'rb')
buf=f.read(4)
magic_num=np.frombuffer(buf,dtype=np.dtype(np.int32).newbyteorder('>'))
if magic_num != 2049:
    print("invalid file ... exiting...")
    exit()
buf=f.read(4)
N=np.frombuffer(buf,dtype=np.dtype(np.int32).newbyteorder('>'))[0]
train_label=np.frombuffer(f.read(N),dtype=np.uint8)
train_label=train_label.reshape(N,1)
f.close()
train_label=np_utils.to_categorical(train_label);

In [3]:
if K.image_data_format() == 'channels_last':
    train_img = train_img.reshape(train_img.shape[0],train_img.shape[1], train_img.shape[2], train_img.shape[3])
else:
    train_img = train_img.reshape(train_img.shape[0], 1, train_img.shape[1], train_img.shape[2])
print(train_img.shape)

(60000, 28, 28, 1)


In [4]:
def new_model():
    model=Sequential()
    model.add(Conv2D(filters=32,kernel_size=(5,5),padding='Valid',activation='relu',input_shape=(28,28,1)));
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Conv2D(filters=12,kernel_size=(5,5),padding='Valid',activation='relu'));
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(10,activation='softmax'))
    return model

def list_sum_vec(a,b):
    for i in range(len(a)):
        a[i]=a[i]+b
    return a
def list_mul_vec(a,b):
    for i in range(len(a)):
        a[i]=a[i]*b
    return a
def list_sub_vec(a,b):
    for i in range(len(a)):
        a[i]=a[i]-b
    return a
def list_div_vec(a,b):
    for i in range(len(a)):
        a[i]=a[i]/b
    return a
def list_sum_list(a,b):
    for i in range(len(a)):
        a[i]=a[i]+b[i]
    return a
def list_mul_list(a,b):
    for i in range(len(a)):
        a[i]=np.multiply(a[i],b[i])
    return a
def list_sub_list(a,b):
    for i in range(len(a)):
        a[i]=a[i]-b[i]
    return a
def list_div_list(a,b):
    for i in range(len(a)):
        a[i]=np.divide(a[i],b[i])
    return a
def list_rand(a):
    for i in range(len(a)):
        a[i]=-1+np.random.random(a[i].shape)*2
    return a
def cross_list(a,b,p):
    c1=a.copy()
    c2=a.copy()
    for i in range(len(a)):
        c1[i]=(a[i]*p)+(b[i]*(1-p))
        c2[i]=(a[i]*(1-p))+(b[i]*p)
    return c1,c2
def list_mutation(a,b):
    for i in range(len(a)):
        a[i]=a[i]+(-1+np.random.random(a[i].shape)*2)*b
    return a

In [5]:
model=new_model();
sgd=optimizers.SGD(lr=0.01,decay=0.0001,momentum=0.9)
model.compile(optimizer=sgd,loss='mean_squared_error',metrics=['accuracy'])
model.fit(train_img,train_label,batch_size=1000,epochs=1,verbose=1)

Epoch 1/1
60000/60000 [==============================] - 7s 116us/step - loss: 0.1538 - acc: 0.2191


In [6]:
pop=[];vel=[];
iter_c=1
pop_sz=4
for i in range(pop_sz):
    m=new_model()
    pop.append(list_rand(m.get_weights()));

obj=[];
for i in range(pop_sz):
    model.set_weights(pop[i])
    v=model.evaluate(train_img,train_label);
    obj.append(v[1])

60000/60000 [==============================] - 3s 54us/step


In [ ]:
iter_c=1
iter_max=50
while iter_c<iter_max:
    prob=np.divide(obj,np.sum(obj))
    pos=np.argsort(np.multiply(obj,-1))
    parent1=pop[pos[1]].copy()
    p1=prob[pos[1]].copy()
    parent2=pop[pos[2]].copy()
    p2=prob[pos[2]].copy()
    temp_pop=pop.copy()
    temp_obj=obj.copy()
    for i in range(0,pop_sz,1):
        if np.random.rand(1)>0.5:
            c1,c2=cross_list(parent1,pop[i],p1)
        else:
            c1,c2=cross_list(parent2,pop[i],p2)
        c1=list_mutation(c1,0.01)
        c2=list_mutation(c2,0.01)
        temp_pop.append(c1)
        model.set_weights(c1)
        v=model.evaluate(train_img,train_label,verbose=0);
        temp_obj.append(v[1])
        temp_pop.append(c2)
        model.set_weights(c2)
        v=model.evaluate(train_img,train_label,verbose=0);
        temp_obj.append(v[1])
    pos=np.argsort(np.multiply(temp_obj,-1))
    for i in range(pop_sz):
        pop[i]=temp_pop[pos[i]].copy()
        obj[i]=temp_obj[pos[i]].copy()
    del temp_pop
    del temp_obj
    print(iter_c,":",np.mean(obj))
    iter_c=iter_c+1

1 : 0.2266375
2 : 0.2277583333333333
3 : 0.22914166666666666
4 : 0.23034583333333333
5 : 0.2318291666666667
6 : 0.23229583333333334
7 : 0.23294583333333335
8 : 0.233375
9 : 0.233875
10 : 0.23511666666666667
11 : 0.2362
12 : 0.23667499999999997
13 : 0.23732083333333331
14 : 0.23840833333333333
15 : 0.238875
16 : 0.23917083333333333
17 : 0.24054166666666668
18 : 0.24062499999999998
19 : 0.24107083333333335
20 : 0.24139166666666664
21 : 0.24148333333333333
22 : 0.24224583333333335
23 : 0.24299166666666666
24 : 0.24362083333333334
25 : 0.24382916666666668
26 : 0.24466249999999998
27 : 0.24562499999999998
28 : 0.24639583333333331
29 : 0.24718333333333334
30 : 0.24827916666666666
31 : 0.24899583333333336
32 : 0.24964583333333332
33 : 0.2504791666666667
34 : 0.2506625
35 : 0.25105833333333333


In [54]:
print()

[12 15 23 20  5  1 11 17 25 27 21  4 18 19 13 26 14 29 28 10  2  9  8  0
 16 24  3  7  6 22]


In [ ]:
agr(max)